In [2]:
import os, pickle
import numpy as np
from scipy.io import loadmat
from dataclasses import dataclass
from typing import List, Dict, Any
import h5py

In [3]:
path = "/home/dacap/Documents/p5control-bluefors-evaluation/simulation/data/Reference"
files = os.listdir(path)

arr = loadmat(f"{path}/{files[10]}")

V = np.array(arr["eV_eff_range"][0], dtype="float64")
I_0 = np.full((len(files) + 1, len(V)), 0, dtype="float64")
I_m = np.full((len(files) + 1, 20, len(V)), 0, dtype="float64")
tau = np.full(len(files) + 1, 0, dtype="float64")
M = np.arange(1, 21)

# arr = loadmat(f'{path}/{files[80]}')


@dataclass
class DataSet:
    V0: np.ndarray
    I0: np.ndarray
    Im: Dict[int, np.ndarray]


sets: Dict[float, DataSet] = {}

for i, file in enumerate(files):
    arr = loadmat(f"{path}/{file}")

    sets[float(file[15:19])] = DataSet(
        V0=np.real(arr["eV_eff_range"][0]),
        I0=np.real(arr["I_In"][:, 0]) / 2,
        Im={
            int(m): np.real(
                arr["In_final"][:, 20 - m] / 2 + arr["In_final"][:, 20 + m] / 2
            )
            for m in M
        },
    )

# save sets as h5py file

with h5py.File("DavidOIV.hdf5", "w") as f:
    for key, dataset in sets.items():
        group = f.create_group(str(key))
        group.create_dataset("V0", data=dataset.V0)
        group.create_dataset("I0", data=dataset.I0)
        for m, Im in dataset.Im.items():
            group.create_dataset(f"Im_{m}", data=Im)

# sorting_indices = np.argsort(tau)
# tau = tau[sorting_indices]
# I_0= I_0[sorting_indices, :]
# I_m = I_m[sorting_indices, :, :]


# V = np.concatenate(
#     (
#     V[:590],
#     -np.linspace(.09, .01, 9),
#     np.linspace(.01, .09, 9),
#     V[590:]
#     ),
#     )

# I_0 = np.concatenate(
#     (
#     I_0[:,:590],
#     np.full((len(files)+1, 18), np.nan),
#     I_0[:,590:]
#     ),
#     axis=1,
#     )

# I_m = np.concatenate(
#     (
#     I_m[:,:,:590],
#     np.full((len(files)+1, 20, 18), np.nan),
#     I_m[:,:,590:]
#     ),
#     axis=2,
#     )

# dIdV = np.gradient(I_0, V, axis=1)
# dIdV_m = np.gradient(I_m, V, axis=2)

# mars = {
#     'V': V,
#     'tau': tau,
#     'I': I_0,
#     'dIdV': dIdV,
#     'm': m,
#     'I_m': I_m,
#     'dIdV_m': dIdV_m,
# }

# with open('DavidOIV.pickle', 'wb') as f:
#     pickle.dump(mars, f)